In [1]:
import requests

In [2]:
url = "https://www.worldometers.info/coronavirus"

In [3]:
req = requests.get(url)

In [4]:
req

<Response [200]>

In [5]:
from lxml.html import fromstring

In [6]:
root = fromstring(req.text)

In [7]:
table = root.xpath(".//table[@id='main_table_countries_today'][1]")[0]

In [8]:
table

<Element table at 0x7efbf82e89f8>

In [9]:
thead = table.xpath(".//thead/tr/th")

In [10]:
def is_visible(attr):
    if 'style' in attr.attrib:
        style = attr.get('style')
        if style.__contains__("display:none") or style.__contains__("display: none"):
            return False
    return True

In [11]:
columns = list()

In [12]:
for th in thead:
    if is_visible(th) is True:
        columns.append(th.text_content().replace('\n', '').replace(u'\xa0', u'').strip())

In [13]:
columns

['Country,Other',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'ActiveCases',
 'Serious,Critical',
 'TotCases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/1M pop']

In [14]:
tbody = table.xpath(".//tbody/tr")

In [15]:
tags = list()
values = list()

In [16]:
for tr in tbody:
    if is_visible(tr) is True:
        tag = tr.xpath(".//td/a")
        if len(tag) > 0:
            tags.append(tag[0].get('href').strip().replace('country/', '').replace('/', ''))

        values.append([value.text_content().strip() for value in tr.xpath(".//td") if is_visible(value) is True])

In [17]:
import pandas as pd

In [18]:
data = pd.DataFrame(values, columns=columns)

In [19]:
data.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,World,"4,120,535","+22,247","280,867",+643,"1,450,884","2,388,784","47,648",529,36.0,,
1,USA,"1,347,318",+9,"80,040",+3,"238,080","1,029,198","16,816","4,070",242,"8,918,345","26,943"
2,Spain,"264,663","+1,880","26,621",+143,"176,439","61,603","1,741","5,661",569,"2,467,761","52,781"
3,Italy,"218,268",,"30,395",,"103,031","84,842","1,034","3,610",503,"2,514,234","41,584"
4,UK,"215,260",,"31,587",,N/A,"183,329","1,559","3,171",465,"1,728,443","25,461"


In [20]:
data.to_csv('../data/overview.csv')